In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install --upgrade pip

In [4]:
# Carica il modulo CUDA 11.7
#!module load cuda/11.7

In [5]:
#!pip install shap

In [6]:
#!pip install numpy --upgrade

In [7]:
#!pip install numpy --upgrade pip

In [8]:
#!pip install pandas --upgrade

In [9]:
#!pip uninstall pandas -y
#!pip install pandas

In [10]:
#!pip install pycocotools

In [11]:
#pip install "numpy<2.0.0"


In [12]:
##Imports
import numpy as np 
import torch
import os, sys, argparse, datetime, shutil
import torchvision.models as models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tensorflow import keras
from tensorflow.python.keras.models import model_from_json
from keras.utils import to_categorical
import shap 
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

#Import from current folder
from utils.config import *
from utils.dataloader import *
from utils.engine import train_one_epoch, evaluate
from utils.train import compute_json_detection
from utils.coco_utils import get_coco_api_from_dataset
from utils.coco_eval import CocoEvaluator
from utils.knowledge_graph import compare_shap_and_KG, reduce_shap, GED_metric, get_bbox_weight
import utils.utils as uti

2024-11-19 22:24:35.071267: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 22:24:35.086413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732051475.104375 1594522 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732051475.109906 1594522 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-19 22:24:35.134502: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [13]:
os.environ["TF_XLA_FLAGS"] = ""

In [14]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [15]:
#os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/usr/local/cuda-11.7"
#os.environ["PATH"] = "/usr/local/cuda-11.7/bin:" + os.environ["PATH"]
#os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-11.7/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")

In [16]:
!which ptxas
!nvcc --version

/usr/bin/which: no ptxas in (/orfeo/cephfs/home/dssc/galessio/jupyter/bin:/usr/local/cuda/bin:/opt/bin/:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin)
/bin/bash: line 1: nvcc: command not found


In [17]:
#pip install tools

In [18]:
sys.path

['/usr/lib64/python39.zip',
 '/usr/lib64/python3.9',
 '/usr/lib64/python3.9/lib-dynload',
 '',
 '/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages',
 '/orfeo/cephfs/home/dssc/galessio/jupyter/lib/python3.9/site-packages',
 '../monumai']

In [19]:
#pip uninstall -y numpy tensorflow

In [20]:
#pip install numpy==1.24 tensorflow

In [21]:
import argparse
import sys
import os

# Imposta il percorso della directory manualmente
os.chdir(os.path.expanduser('~/XAI_Monuments/tools'))
notebook_dir = os.getcwd()
sys.path.append(notebook_dir)

from pickle_tools import *

TMP_TRAIN = TMP_PATH + '/train'
TMP_VAL = TMP_PATH + '/val'
TMP_TEST = TMP_PATH + '/test'

os.makedirs(TMP_VAL, exist_ok=True)
os.makedirs(TMP_TRAIN, exist_ok=True)
os.makedirs(TMP_TEST, exist_ok=True)

# Argparse
parser = argparse.ArgumentParser(description='Arguments needed to prepare the metadata files')
parser.add_argument('--resume', dest='resume', help='Whether or not to resume a training', default=False)
parser.add_argument('--path_resume', dest='path_resume', help='Path to the model to load', default='./model/model_monumenai.pth')
parser.add_argument('--epoch_classif', dest='epoch_classif', help='Number of epochs to train the classification model', default=150)
parser.add_argument('--batch_size', dest='batch_size', help='Batch size to train the classification model', default=8)
parser.add_argument('--neuron_classif', dest='neuron_classif', help='Number of neurons in the classification model', default=11)
parser.add_argument('--epoch_detection', dest='epoch_detection', help='Number of epochs to train the detection model', default=0)
parser.add_argument('--lr', dest='lr', help='Learning rate of the detection model', default=0.0003)
parser.add_argument('--stepLR', dest='stepLR', help='Step of the learning rate scheduler', default=9)
parser.add_argument('--gammaLR', dest='gammaLR', help='Gamma parameter of the learning rate scheduler', default=0.1)
parser.add_argument('--weight', dest='weight', help='Type of weighting', default='None')
parser.add_argument('--exp_weights', dest='exp_weights', help='linear or exponential weighting', default='linear')
parser.add_argument('--data', dest='data', help='MonumenAI or PascalPart', default='MonumenAI')

# Se stai usando un ambiente Jupyter, ignora gli argomenti aggiuntivi
if 'ipykernel_launcher' in sys.argv[0]:
    sys.argv = sys.argv[:1]

args = parser.parse_args()

In [22]:
#Hyperparameters classification
n_neurons_classification = int(args.neuron_classif)
num_epochs_classification = int(args.epoch_classif)
batch_size_classification = int(args.batch_size)
learning_rate_classification = None

#Hyperparameters detection
args.epoch_detection = 150
num_epochs_detection = int(args.epoch_detection)
learning_rate_detection = float(args.lr)
stepLR = float(args.stepLR)
gammaLR = float(args.gammaLR)
data = args.data

In [23]:
import os
import argparse
import sys
os.chdir(os.path.expanduser('~/XAI_Monuments/tools'))
notebook_dir = os.getcwd()
sys.path.append(notebook_dir)
if data == 'MonumenAI':
    from metadata_tools import *
    from monumai.monument import Monument
    archi_features = [el for sublist in list(Monument.ELEMENT_DIC.values()) for el in sublist]
    styles = FOLDERS_DATA
    #Loaders for detection
    PATH_DATA = "../"+PATH_DATA
    train_loader = ArchitectureDetectionDataset(os.path.join(PATH_DATA, 'train.csv'), os.path.join(PATH_DATA, CSV_XML), transform_detection)

    if num_epochs_detection != 0:
        val_loader = ArchitectureDetectionDataset(os.path.join(PATH_DATA,'val.csv'), os.path.join(PATH_DATA, CSV_XML), transform_detection)
    else:
        #Actually loading the test set
        val_loader = ArchitectureDetectionDataset(os.path.join(PATH_DATA,'val.csv'), os.path.join(PATH_DATA, CSV_XML), transform_detection)
        test_loader = ArchitectureDetectionDataset(os.path.join(PATH_DATA,'test.csv'), os.path.join(PATH_DATA, CSV_XML), transform_detection)
if data == 'PascalPart':
    from tools.metadata_tools_pascal import *
    from monumai.pascal import Monument
    archi_features = [el for sublist in list(Monument.ELEMENT_DIC.values()) for el in sublist]
    styles = list(PASCAL_EL_DIC.keys())
    #Loaders for detection
    train_loader = PascalDetectionDataset(train_pascal, PATH_PASCAL+PASCAL_IMG,PATH_PASCAL+PASCAL_XML, transform_detection_pascal)
    if num_epochs_detection != 0:
        val_loader = PascalDetectionDataset(val_pascal, PATH_PASCAL+PASCAL_IMG,PATH_PASCAL+PASCAL_XML, transform_detection_pascal)
    else:
        #Actually loading the test set
        val_loader = PascalDetectionDataset(val_pascal, PATH_PASCAL+PASCAL_IMG,PATH_PASCAL+PASCAL_XML, transform_detection_pascal)
        test_loader = PascalDetectionDataset(test_pascal, PATH_PASCAL+PASCAL_IMG,PATH_PASCAL+PASCAL_XML, transform_detection_pascal)

In [24]:
##Hyperparameters for detection

num_archi_features = len(archi_features)
num_classes_detection = num_archi_features + 1  # num_archi_features + background
num_styles = len(styles)

In [25]:
##Build detection model
if args.weight == "bbox_level":
    from utils.pytorch_utils import fasterrcnn_resnet50_fpn_custom
    detector = fasterrcnn_resnet50_fpn_custom(True)
    in_features = detector.roi_heads.box_predictor.cls_score.in_features
    detector.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes_detection)
else:
    detector = models.detection.fasterrcnn_resnet50_fpn(True)
    in_features = detector.roi_heads.box_predictor.cls_score.in_features
    detector.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes_detection)

if args.exp_weights == 'exponential':
    is_exponential = True
elif args.exp_weights == 'linear':
    is_exponential = False
else:
    print("Unrecognized type of weighting, defaulted to linear")
    is_exponential = False


/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
import torch
import torchvision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
detector.to(device)

optimizer = torch.optim.SGD(detector.parameters(), lr=learning_rate_detection, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, stepLR, gamma=gammaLR, last_epoch=-1)

if args.resume:
    detector.load_state_dict(torch.load(args.path_resume, map_location=device))

/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(


In [27]:
import time, json, os
def compute_json_detection(detector, loader, path, dataset = 'MonumenAI'):
    #Run inference on all data to prepare for classification
    detector.eval()
    if dataset == 'MonumenAI':
        information_about_class = ['M', 'G', 'R', 'B']
        elemen_dic = SUB_ELEMENTS
        reverse_dic = SUB_ELEMENTS_REVERSED
    if dataset == 'PascalPart':
        information_about_class = list(PASCAL_EL_DIC.keys())
        elemen_dic = PASCAL_PART_DIC
        reverse_dic = PASCAL_PART_DIC_REVERSED
    for k in range(len(loader)):
        print(f"{k/len(loader)*100}", end='\r')
        img = loader[k][0][0].cuda()
        results = detector([img])[0]
        r_b = results['boxes'].detach().cpu().numpy()
        scores = results['scores'].detach().cpu().numpy()
        classes = results['labels'].detach().cpu().numpy()
        unique, counts = np.unique(classes, return_counts=True)
        counter = dict(zip(unique, counts))

        if dataset == 'MonumenAI':
            img_name = loader.images_loc.iloc[k, 0]
        if dataset == 'PascalPart':
            img_name = loader.images_loc[k]

        condensced_results = {}
        condensced_results["num_predictions"] = []
        condensced_results["image"] = img_name
        condensced_results["object"] = []
        if dataset == 'MonumenAI':
            condensced_results["true_label"] = int(loader.images_loc.iloc[k, 1])
        if dataset == 'PascalPart':
            condensced_results["true_label"] = get_label(loader.xml_loc[k], PASCAL_EL_DIC)

        for name in elemen_dic:

            if elemen_dic[name] in counter:
                condensced_results["num_predictions"].append({
                    name :  int(counter[elemen_dic[name]])
                })
            else:
                condensced_results["num_predictions"].append({
                    name :  0
                })
            
        for k in range(len(r_b)):
            if classes[k] in reverse_dic:
                box = r_b[k]/224.
                local_result = {
                    "bndbox" : {
                        "xmin": str(box[0]),
                        "ymin": str(box[1]),
                        "ymax": str(box[3]),
                        "xmax": str(box[2])
                    },
                    "score" : str(scores[k]),
                    "class" : reverse_dic[classes[k]]
                }
                condensced_results["object"].append(local_result)
        local_path = os.path.join(path,information_about_class[condensced_results["true_label"]] + '_' + img_name.split('/')[-1][:-4] + '.json')
        with open(local_path, 'w') as fp:
            json.dump(condensced_results, fp)

In [28]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical

# Configurazione directory checkpoint
CHECKPOINT_DIR = "./checkpoints2/"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Funzione per caricare il checkpoint più recente
def load_checkpoint():
    checkpoints = [f for f in os.listdir(CHECKPOINT_DIR) if f.startswith("detector")]
    if checkpoints:
        last_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[2].split('.')[0]))
        checkpoint_path = os.path.join(CHECKPOINT_DIR, last_checkpoint)
        checkpoint = torch.load(checkpoint_path)
        print(f"Checkpoint caricato: {checkpoint_path}")
        return checkpoint
    return None

checkpoint = load_checkpoint()
start_epoch = checkpoint['epoch'] + 1 if checkpoint else 0

# Caricamento del modello e dello stato ottimizzatore
if checkpoint:
    detector.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

# Funzione per preparare i dati per il classificatore
def prepare_data(loader, TMP, dataset):
    matrix_metadata = metadata_to_matrix(TMP, "json")
    names = matrix_metadata[:, -1]
    data = np.zeros((len(names), num_archi_features))
    labels = np.zeros(len(names))

    if dataset == "MonumenAI":
        for i, name in enumerate(names):
            im_name = name[2:-4]
            idx = loader.images_loc['path'].str.contains(im_name)
            data[idx] = matrix_metadata[i, :num_archi_features]
            labels[idx] = matrix_metadata[i, num_archi_features]
    elif dataset == "PascalPart":
        for i, name in enumerate(names):
            im_name = os.path.join(PATH_PASCAL + PASCAL_IMG, name.split('_')[1][:-5] + '.jpg')
            idx = loader.images_loc.index(im_name)
            data[idx] = matrix_metadata[i, :num_archi_features]
            labels[idx] = matrix_metadata[i, num_archi_features]

    data = data.astype(np.float32)
    labels = to_categorical(labels.astype(np.float32).astype(np.int8))
    return data, labels

# Loop per il training del detector e del classificatore
for epoch in range(start_epoch, num_epochs_detection + 1):
    print('------------------')
    print(f"Epoch {epoch + 1}/{num_epochs_detection}")

    if epoch > 0 or args.resume:
        detector.eval()
        # Inference per la preparazione dei dati
        compute_json_detection(detector, train_loader, TMP_TRAIN, dataset=data)
        compute_json_detection(detector, val_loader, TMP_VAL, dataset=data)

        print('Inference completata')

        # Preparazione dei dati
        train_data, train_label = prepare_data(train_loader, TMP_TRAIN, data)
        test_data, test_label = prepare_data(val_loader, TMP_VAL, data)

        # Classificatore
        classificator = keras.Sequential([
            keras.layers.Dense(units=n_neurons_classification, activation='relu', input_shape=(num_archi_features,)),
            keras.layers.Dense(units=num_styles, activation='softmax')
        ])
        classificator.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        print("Training del classificatore...")
        history = classificator.fit(
            train_data, train_label,
            batch_size=batch_size_classification,
            epochs=num_epochs_classification,
            verbose=1
        )
        loss, accuracy = classificator.evaluate(test_data, test_label, verbose=1)
        print(f"Loss: {loss}, Accuracy: {accuracy}")

        # SHAP Analysis
        print("Calcolo degli SHAP values...")
        elements = np.random.choice(len(train_data), int(0.3 * len(train_data)), replace=False)
        explainer = shap.KernelExplainer(classificator.predict, train_data[elements])

        shap_values_test = explainer.shap_values(test_data, nsamples=30)
        print("SHAP values calcolati.")
        for i, shap_val in enumerate(shap_values_test):
            print(f"SHAP values per classe {i}: {shap_val[:5]}")  # Stampa i primi valori per debug

        d = GED_metric(test_data, shap_values_test, dataset=data)
        print(f"SHAP GED: {d}")

        classificator.save_weights(os.path.join(CHECKPOINT_DIR, f"classificator_epoch_{epoch}.weights.h5"))

    if epoch < num_epochs_detection:
        # Training del detector
        detector.train()
        print("Training del detector...")
        for images, targets in train_loader:
            images = [img.to('cuda') for img in images]
            targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]

            loss_dict = detector(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

        scheduler.step()

        # Salvataggio del checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': detector.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
        }, os.path.join(CHECKPOINT_DIR, f"detector_epoch_{epoch}.pth"))
        print(f"Checkpoint salvato per l'epoca {epoch}.")

# Valutazione finale
if num_epochs_detection == 0:
    loss, accuracy = classificator.evaluate(test_data, test_label, verbose=1)
    print(f"Loss finale: {loss}, Accuracy finale: {accuracy}")

    predict_test = classificator.predict(test_data)
    prediction = np.argmax(predict_test, axis=1)
    true_labels = np.argmax(test_label, axis=1)

    cm = confusion_matrix(true_labels, prediction)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=STYLES_HOTONE_ENCODE.keys())
    disp.plot(include_values=True, cmap='viridis')
    plt.savefig('confmatDeLDECAS.png')

    shap_values_test = explainer.shap_values(test_data, nsamples=30)
    d = GED_metric(test_data, shap_values_test, dataset=data)
    print(f"Final SHAP GED: {d}")

Checkpoint caricato: ./checkpoints2/detector_epoch_8.pth
------------------
Epoch 10/150


/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Attempt to open cnn_infer failed: handle=0 error: libcudnn_cnn_infer.so.8: cannot open shared object file: No such file or directory (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:81.)
  return F.conv2d(input, weight, bias, self.stride,


/orfeo/cephfs/home/dssc/galessio/jupyter/lib/python3.9/site-packages/albumentations/core/bbox_utils.py:476: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)


Inference completata
Load .json metadata:  ./tmp/train/B_20181212_110313.json
Load .json metadata:  ./tmp/train/B_5bed71eae40c4.json
Load .json metadata:  ./tmp/train/B_20181212_110321.json
Load .json metadata:  ./tmp/train/M_puerta_mihrab_mezquita_cordoba.json
Load .json metadata:  ./tmp/train/B_5bad4cbc18c44.json
Load .json metadata:  ./tmp/train/R_5c1f82413637d.json
Load .json metadata:  ./tmp/train/G_5c67d991a4cf7.json
Load .json metadata:  ./tmp/train/B_5dd7d6e26f601.json
Load .json metadata:  ./tmp/train/M_alcazaba_malaga3.json
Load .json metadata:  ./tmp/train/G_casa_bailio_cordoba.json
Load .json metadata:  ./tmp/train/G_5babc82fc022f.json
Load .json metadata:  ./tmp/train/M_5e4fb912bf15b.json
Load .json metadata:  ./tmp/train/R_5c75195fbb8f2.json
Load .json metadata:  ./tmp/train/G_5bb8f33edfa1f.json
Load .json metadata:  ./tmp/train/M_5bb0d93bcd653.json
Load .json metadata:  ./tmp/train/B_5c15322219e6f.json
Load .json metadata:  ./tmp/train/B_facultad_derecho_granada2.json
Lo

/orfeo/cephfs/home/dssc/galessio/jupyter/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1732051807.268201 1594522 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


...

SHAP values calcolati.
SHAP values per classe 0: [[-0.13835954  0.12031355  0.          0.        ]
 [-0.00338533  0.01210082 -0.00475813  0.        ]
 [-0.03334361  0.02633653  0.          0.        ]
 [-0.03999264  0.25186206 -0.09222849 -0.08212858]
 [ 0.          0.          0.          0.        ]]
SHAP values per classe 1: [[-0.08806295  0.02943569 -0.06036968  0.09265575]
 [-0.00890722  0.          0.          0.        ]
 [-0.06508922 -0.07105502  0.21061061  0.        ]
 [ 0.         -0.06466121  0.03179882  0.        ]
 [ 0.         -0.01589045  0.          0.        ]]
SHAP values per classe 2: [[-0.07126073  0.03372392  0.          0.13482955]
 [ 0.         -0.02512705  0.          0.0225337 ]
 [-0.00462735  0.00870467  0.          0.        ]
 [-0.01879892 -0.13315922  0.         -0.04067404]
 [ 0.          0.          0.          0.        ]]
SHAP values per classe 3: [[-0.13246851 -0.09591698  0.          0.        ]
 [-0.0141811   0.          0.          0.        ]
 [-

Training del detector...


/orfeo/cephfs/home/dssc/galessio/jupyter/lib/python3.9/site-packages/albumentations/core/bbox_utils.py:476: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)


Checkpoint salvato per l'epoca 12.
------------------
Epoch 14/150


KeyboardInterrupt: 

In [ ]:
shutil.rmtree(TMP_PATH)